# Профилирование и оптимизация выполнения кода

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* IPython Cookbook, Second Edition (2018), глава 4
* https://ipython-books.github.io/43-profiling-your-code-line-by-line-with-line_profiler/

## Задачи для совместного разбора

In [ ]:
# !pip install line_profiler
# !pip install --user numpy==1.20

In [1]:
import numpy as np
import pandas as pd

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
A=np.random.randint(0,1001, size=1_000_000)
A


array([223, 441, 448, ..., 779, 202, 603])

In [3]:
def slow(A):
    acc = 0
    for a in A:
        b=a+100
        acc +=b
    return acc/len(A)

In [4]:
%time
slow(A)

Wall time: 0 ns


599.648203

In [5]:
%%time
(A+100).mean()

Wall time: 2 ms


599.648203

In [6]:
%%time
def fast(A):
    cnt=len(A)
    s=sum(A)+100*cnt
    return(s)
fast(A)

Wall time: 70 ms


599648203

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [7]:
%load_ext line_profiler

In [8]:
import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,0.212640,0.970420,-1.149077,-0.922875,w
1,0.733322,-2.160399,0.050225,1.010148,w


## Лабораторная работа 1

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

In [28]:
recipes =  pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')
reviews

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [29]:
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [30]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
recipes =recipes[(recipes['submitted'] > '01-01-2010') & (recipes['submitted'] <= '31-12-2010')]
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0
...,...,...,...,...,...,...,...,...
29897,zoe s chicken tarragon,441211,40,76559,2010-11-04,12.0,from a good housekeeping at my hair salon. ha...,9.0
29907,zucchini and noodle slice,412518,60,423555,2010-02-10,21.0,"a yummy, tasty slice packed with vegies and ri...",13.0
29915,zucchini bread bread machine,409757,220,539686,2010-01-22,7.0,"originally from a packet of red star yeast, th...",NaN
29926,zucchini chip cupcakes,406686,35,628076,2010-01-04,9.0,this is a great tasting recipe to use up zucch...,14.0


In [32]:
recipes['len']=recipes['name']+' '+recipes['description']
recipes

<ipython-input-32-15f900fce167>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes['len']=recipes['name']+' '+recipes['description']


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,len
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0,just peachy cobbler all i can say is yummmmmm...
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0,the heat spicy party mix a spicy chex mix tha...
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN,iowa state fair sweet dough caramel cinnamon ...
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN,1 minute blueberries cream i was craving blu...
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0,2 2 2 diet mocha while trying to come up with ...
...,...,...,...,...,...,...,...,...,...
29897,zoe s chicken tarragon,441211,40,76559,2010-11-04,12.0,from a good housekeeping at my hair salon. ha...,9.0,zoe s chicken tarragon from a good housekeepin...
29907,zucchini and noodle slice,412518,60,423555,2010-02-10,21.0,"a yummy, tasty slice packed with vegies and ri...",13.0,"zucchini and noodle slice a yummy, tasty slice..."
29915,zucchini bread bread machine,409757,220,539686,2010-01-22,7.0,"originally from a packet of red star yeast, th...",NaN,zucchini bread bread machine originally from...
29926,zucchini chip cupcakes,406686,35,628076,2010-01-04,9.0,this is a great tasting recipe to use up zucch...,14.0,zucchini chip cupcakes this is a great tasting...


## Измерение времени выполнения кода

Создайте версию таблицы, содержащие строки строки для рецептов, которые были добавлены в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году. Полным описанием рецепта называется строка, полученная путем конкатенации названия и описания рецепта через пробел.

№1\.1 С использованием метода `DataFrame.iterrows` таблицы:

- функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
- вычисление полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [34]:
#Черновик
def get_mean_len_A(df: pd.DataFrame) -> float:
    lens=[]
    for i,row in df.iterrows():
        lens.append(len(str(row['len'])))
    res = np.mean(lens)
    return res
get_mean_len_A(recipes)

265.751953125

In [35]:
def get_mean_len_A(df: pd.DataFrame) -> float:
    lens = []
    for i in df.iterrows():
        lens.append(len((i[1]['name']) + ' ' + (i[1]['description'])))
        
    return np.mean(lens)
        
get_mean_len_A(recipes)

265.751953125

№1\.2. С использованием метода `DataFrame.apply` таблицы:

- функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
- вызываете метод apply у таблицы; в качестве аргумента передаете функцию, которая возвращает длину полного описания для каждой строки;
    
- считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [39]:
def get_mean_len_B(df: pd.DataFrame) -> float:
    def get_full_desc(s: pd.Series) -> str:
        return str(s['name']) + ' ' + str(s['description'])
    
    return df.apply(get_full_desc, axis=1).str.len().mean()
        
get_mean_len_B(recipes)

265.751953125

№1\.3. С использованием векторизованных методов серий `pd.Series`:

- функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
- при помощи векторизованной операции сложения получаете столбец с полным описанием;
    
- считаете длину каждого элемента столбца с полным описанием, воспользовавшись соответствующим строковым методом аксессора `.str`;
    
- считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [40]:
def get_mean_len_C(df: pd.DataFrame) -> float:
    return (df['name'] + ' ' + df['description']).str.len().mean()

get_mean_len_C(recipes)

265.751953125

№1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [42]:
get_mean_len_A(recipes) == get_mean_len_B(recipes) == get_mean_len_C(recipes)

True

In [43]:
%%time #или timeit

get_mean_len_A(recipes)

Wall time: 77 ms


265.751953125

In [44]:
%%time

get_mean_len_B(recipes)

Wall time: 18 ms


265.751953125

In [45]:
%%time

get_mean_len_C(recipes)

Wall time: 2 ms


265.751953125

In [46]:
77/2

38.5

## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [83]:
import re


def get_word_reviews_count(df):
    word_reviews = {}
    for review_id, row in df.dropna(subset=["review"]).iterrows(): 
        review = row["review"]
        words = re.sub(r"[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word.lower() not in word_reviews:
                word_reviews[word.lower()] = set()
            word_reviews[word.lower()].add(review_id)
    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub(r"[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word.lower()] = len(word_reviews[word.lower()])
    return word_reviews_count
 #23.763    0.000 frame.py:1026(iterrows)
# 1    0.076    0.076   39.213   39.213 <string>:1(<module>)
#1    0.000    0.000   39.213   39.213 {built-in method builtins.exec}

№2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [96]:
inf = %prun -r get_word_reviews_count(reviews)
inf

№2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [85]:
def get_word_reviews_count_optimized(df):
    word_reviews_count = {}
    revs = df.review.dropna().apply(lambda x: re.sub('[^A-Za-z\s]', '', x)).str.split()
    for rev in revs:
        for word in rev:
            if word not in word_reviews_count.keys():
                word_reviews_count[word] = 0
            word_reviews_count[word] += 1
                
    return word_reviews_count

In [94]:
%%time 
get_word_reviews_count(reviews)


Wall time: 19.4 s


{'last': 4517,
 'week': 1489,
 'whole': 5540,
 'sides': 435,
 'of': 61867,
 'frozen': 2722,
 'salmon': 819,
 'fillet': 86,
 'was': 56972,
 'on': 28791,
 'sale': 255,
 'in': 43940,
 'my': 44544,
 'local': 565,
 'supermarket': 93,
 'so': 39441,
 'i': 101329,
 'bought': 1490,
 'tons': 184,
 'okay': 717,
 'only': 13679,
 '': 89125,
 'but': 36936,
 'total': 557,
 'weight': 290,
 'over': 8762,
 'pounds': 275,
 'this': 83593,
 'recipe': 54531,
 'is': 41236,
 'perfect': 8643,
 'for': 75829,
 'even': 8881,
 'though': 4791,
 'it': 73971,
 'calls': 513,
 'steaks': 434,
 'cut': 6416,
 'up': 14352,
 'the': 95894,
 'into': 6364,
 'individual': 304,
 'portions': 209,
 'and': 97007,
 'followed': 5450,
 'instructions': 971,
 'exactly': 4678,
 'im': 7768,
 'one': 15973,
 'those': 2408,
 'food': 3473,
 'combining': 82,
 'diets': 45,
 'left': 4958,
 'out': 22223,
 'white': 3493,
 'wine': 1580,
 'added': 19387,
 'just': 23483,
 'a': 84192,
 'dash': 617,
 'vinegar': 1641,
 'instead': 11221,
 'little': 14634

In [95]:
%%time 
get_word_reviews_count_optimized(reviews) #в 8-9 раз быстрее


Wall time: 2.79 s


{'Last': 104,
 'week': 1517,
 'whole': 5787,
 'sides': 465,
 'of': 109251,
 'frozen': 2946,
 'salmon': 1035,
 'fillet': 92,
 'was': 89646,
 'on': 36154,
 'sale': 263,
 'in': 62824,
 'my': 44666,
 'local': 572,
 'supermarket': 95,
 'so': 46871,
 'I': 291277,
 'bought': 1518,
 'tons': 163,
 'okay': 603,
 'only': 14218,
 'but': 43054,
 'total': 515,
 'weight': 216,
 'over': 9813,
 'pounds': 292,
 'This': 40046,
 'recipe': 72202,
 'is': 56553,
 'perfect': 7873,
 'for': 123210,
 'even': 8077,
 'though': 4968,
 'it': 133661,
 'calls': 525,
 'steaks': 514,
 'cut': 6849,
 'up': 16214,
 'the': 266580,
 'into': 7089,
 'individual': 314,
 'portions': 216,
 'and': 219293,
 'followed': 4882,
 'instructions': 1010,
 'exactly': 4634,
 'Im': 8420,
 'one': 17665,
 'those': 2419,
 'food': 3517,
 'combining': 78,
 'diets': 46,
 'left': 5075,
 'out': 25734,
 'white': 3696,
 'wine': 1769,
 'added': 22119,
 'just': 25284,
 'a': 166494,
 'dash': 547,
 'vinegar': 1957,
 'instead': 11664,
 'little': 16811,
 'b